#### Define Imports

In [87]:
import requests
import pandas as pd 
import random

api_key = '5TjgNMFCh7h44T09HbQnbGhU8as11D0FDdjfJhgV'
api_base_url = 'https://developer.nps.gov/api/v1/'

from helper_functions import *
state_model, state_vectorizer = state_code_model.trained_model()
park_model, park_vectorizer = park_code_model.trained_model()

#### Define Functions

In [88]:
def get_parks(params):
    """
    Use to find a list of all park names, codes, states, addresses and descriptions from the NPS parks endpoint.
    Can also be used to find specific park information.

    """
    parks = []
    limit = 50  # Number of results per page, maximum allowed by NPS API
    start = 0   # Initial starting point for pagination
    
    while True:
        params = {
            'api_key': api_key,
            'limit': limit,
            'start': start
        }
        
        response = requests.get(f"{api_base_url}parks", params=params)
        data = response.json()
        
        parks.extend([
            {
                'fullName': park['fullName'],
                'parkCode': park['parkCode'],
                'state': park['states'],
                'addresses': park.get('addresses', []),
                'description': park['description']
            } for park in data['data']
        ])
        
        # Move to the next page
        start += limit
        
        # Break the loop if all parks have been retrieved
        if int(start) >= int(data['total']):
            break
    
    return parks

In [89]:
def get_parks_in_state(params):
    """
    Retrieve a list of parks in a specified state.
    
    state_code: The code of the state (e.g., 'CA' for California)
    api_key: Personal API key to use in request
    """
    parks_in_state = []
    
    response = requests.get(f"{api_base_url}parks", params=params)
    data = response.json()
    
    for park in data['data']:
        if params['stateCode'] in park['states'].split(','):
            parks_in_state.append(park['fullName'])
    
    return parks_in_state

In [138]:
def get_basic(endpoint, params):
    """
    Use to get all data from endpoint without specific processing

    endpoint: The API endpoint to call
    params: The param dict to pass through the API call
    """
    responses = []
    limit = 50  # Number of results per page, maximum allowed by NPS API
    start = 0   # Initial starting point for pagination
    
    while True:
        params['limit'] = limit
        params['start'] =  start
        
        request = requests.get(f"{api_base_url}{endpoint}", params=params)
        request_data = request.json()

        for record in request_data['data']:
            responses.extend([record])
        
        # Move to the next page
        start += limit
        
        # Break the loop if all responses have been retrieved
        if int(start) >= int(request_data['total']):
            break

    return responses

In [161]:
def get_info(api_key, entities, entityCode, endpoint, intent, queries, response_call=0):
    """
    Creates synthetic data in the necessary format for a specified API call.

    api_key: Personal API key to use in request.
    entities: List of items to loop through such as State, Parks, Amentities.
    endpoint: the NPS API endpoint to call such as /activities or /parks.
    intent: General label for queries in a particular group. 
            For example, the questions "Tell me about {park}" and "I want to know more about {park}" could both be categorized with the label "GetParkInfo".
    queries: A list of queries you would like to associate with a given set of API calls.
    entityCode: The entity code to be used in the API call. e.g. stateCode & parkCode
        *This will likely need to be updated to have more dynamic functionality.
    response_call: Pass through the function name that should be used to intiate the API call. The results will be recorded in the response column of the dataset in list format.
    """
    dataset = []
    for entity in entities:
        for query in queries:
            # Create API parameters 
            if entityCode == "parkCode":
                # Park name needs to be converted to park code
                params = {'api_key': api_key,
                        entityCode: park_code_model.map_park_code(query.format(entity=entity), park_model, park_vectorizer)
                }
            if entityCode == "stateCode":
                # State name needs to be converted to state code
                params = {'api_key': api_key,
                        entityCode: state_code_model.map_state_code(query.format(entity=entity), state_model, state_vectorizer)
                }
                
            # Set response function to use
            if response_call == 0:
                response = ""
            elif intent == "ParkInfo":
                response = get_parks(params)
            elif intent == "ParksInState":
                response = get_parks_in_state(params)
            else: 
                response = get_basic(endpoint, params)

            dataset.append({
                "query": query.format(entity=entity),
                "intent": intent,
                "api_call": {
                    "endpoint": endpoint,
                    entityCode: params[entityCode]
                },
                "response":response
            })

    synthetic_queries = pd.json_normalize(dataset)
    return synthetic_queries


#### Define Variables

In [150]:

categories = ['activities', 'activities/parks', 'alerts', 'amenities','amenities/parksvisitorcenters',
              'amenities/parksplaces', 'articles', 'campgrounds', 'events', 'feespasses', 
              'lessonplans', 'multimedia/audio', 'multimedia/galleries', 'newsreleases',
              'parkinglots', 'parks', 'places', 'people', 'thingstodo', 
              'topics', 'topics/parks', 'tours', 'visitorcenters', ]


# list of parks
parks_df = pd.DataFrame(get_parks({'api_key': api_key}))
parks = parks_df['fullName'].tolist()
park_codes = parks_df['parkCode'].tolist()
park_lookup = dict(zip(parks, park_codes))
park_roots = nps_parks_root.nps_parks_root()
# Parks combined is the combination of two lists: The full park names and estimated park name abbreviations that users might use (i.e. Acadia National Park vs Acadia)
parks_combined = park_roots+parks

dist_states = ["Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", 
                "Connecticut", "Delaware", "Florida", "Georgia", "Hawaii", "Idaho", 
                "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana", 
                "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota", 
                "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", 
                "New Hampshire", "New Jersey", "New Mexico", "New York", 
                "North Carolina", "North Dakota", "Ohio", "Oklahoma", "Oregon", 
                "Pennsylvania", "Rhode Island", "South Carolina", "South Dakota", 
                "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington", 
                "West Virginia", "Wisconsin", "Wyoming"]

#### Create Synthetic Data

In [169]:
parks = ['acadia']
queries = ["Where can I park at {entity}?"]
ParkParkingLots = get_info(api_key, entities = parks, entityCode = "parkCode", endpoint = "parkinglots", intent = "ParkParkingLots", queries = queries, response_call = 0)
ParkParkingLots

,query,intent,response,api_call.endpoint,api_call.parkCode
0,Where can I park at acadia?,ParkParkingLots,[{'id': '7DE234F8-0156-4ADE-BB94-CA01B72DA7C2'...,parkinglots,acad


In [162]:
queries = ["Tell me about {entity}"]
ParkInfo = get_info(api_key, entities = parks_combined, entityCode = "parkCode", endpoint = "parks", intent = "ParkInfo", queries = queries, response_call = 0)

queries = ["Which parks are in {entity}?"]
ParksInState = get_info(api_key, entities = dist_states, entityCode = "stateCode", endpoint = "parks", intent = "ParksInState", queries = queries, response_call = 0)

queries = ["What activities can I do in {entity}"]
ParkActivities = get_info(api_key, entities = parks_combined, entityCode = "parkCode", endpoint = "activities", intent = "ParkActivities", queries = queries, response_call = 0)

queries = ["What alerts are active at {entity} currently?"]
ParkAlerts = get_info(api_key, entities = parks_combined, entityCode = "parkCode", endpoint = "alerts", intent = "ParkAlerts", queries = queries, response_call = 0)

queries = ["What activities can I do at {entity}?"]
ParkAmenities = get_info(api_key, entities = parks, entityCode = "parkCode", endpoint = "amenities", intent = "ParkAmenities", queries = queries, response_call = 0)

queries = ["What events are happening at {entity}?"]
ParkEvents = get_info(api_key, entities = parks, entityCode = "parkCode", endpoint = "events", intent = "ParkEvents", queries = queries, response_call = 0)

queries = ["How much does it cost to get into {entity}?"]
ParkFees = get_info(api_key, entities = parks, entityCode = "parkCode", endpoint = "feespasses", intent = "ParkFees", queries = queries, response_call = 0)

queries = ["Where can I park at {entity}?"]
ParkParkingLots = get_info(api_key, entities = parks, entityCode = "parkCode", endpoint = "parkinglots", intent = "ParkParkingLots", queries = queries, response_call = 0)

synthetic_queries_df = pd.concat([ParkInfo, ParkActivities, ParksInState, ParkAlerts, ParkAmenities, ParkEvents, ParkFees, ParkParkingLots], axis=0, ignore_index=True)

In [152]:
synthetic_queries_df[synthetic_queries_df['api_call.parkCode'] == "iatr"] 

,query,intent,response,api_call.endpoint,api_call.parkCode,api_call.stateCode
229,Tell me about Ice Age,GetParkInfo,,parks,iatr,NaN
700,Tell me about Ice Age National Scenic Trail,GetParkInfo,,parks,iatr,NaN
1171,What activities can I do in Ice Age,GetParkActivities,,activities,iatr,NaN
1642,What activities can I do in Ice Age National S...,GetParkActivities,,activities,iatr,NaN
2163,What alerts are active at Ice Age currently?,GetAlertsInPark,,alerts,iatr,NaN
2634,What alerts are active at Ice Age National Sce...,GetAlertsInPark,,alerts,iatr,NaN
3105,What activities can I do at Ice Age?,GetAmenitiesInPark,,amenities,iatr,NaN
3576,What activities can I do at Ice Age National S...,GetAmenitiesInPark,,amenities,iatr,NaN


In [153]:
synthetic_queries_df[synthetic_queries_df['intent'] == "FindParksInState"] 

,query,intent,response,api_call.endpoint,api_call.parkCode,api_call.stateCode
1884,Which parks are in Alabama?,FindParksInState,"[Birmingham Civil Rights National Monument, Fr...",parks,NaN,AL
1885,Which parks are in Alaska?,FindParksInState,"[Alagnak Wild River, Alaska Public Lands, Aleu...",parks,NaN,AK
1886,Which parks are in Arizona?,FindParksInState,"[Butterfield Overland National Historic Trail,...",parks,NaN,AZ
1887,Which parks are in Arkansas?,FindParksInState,"[Arkansas Post National Memorial, Buffalo Nati...",parks,NaN,AR
1888,Which parks are in California?,FindParksInState,"[Alcatraz Island, Butterfield Overland Nationa...",parks,NaN,CA
1889,Which parks are in Colorado?,FindParksInState,"[Amache National Historic Site, Bent's Old For...",parks,NaN,CO
1890,Which parks are in Connecticut?,FindParksInState,"[Appalachian National Scenic Trail, Coltsville...",parks,NaN,CT
1891,Which parks are in Delaware?,FindParksInState,[Captain John Smith Chesapeake National Histor...,parks,NaN,DE
1892,Which parks are in Florida?,FindParksInState,"[Big Cypress National Preserve, Biscayne Natio...",parks,NaN,FL
1893,Which parks are in Georgia?,FindParksInState,"[Andersonville National Historic Site, Appalac...",parks,NaN,GA


#### Development

In [90]:
def get_activities_list(params):
    """
    Use to get a list of activities available at a specific park based on park code
    """

    # Define the endpoint for the activities query
    activities_endpoint = f"{api_base_url}activities/parks"
    
    # get activities information
    response = requests.get(activities_endpoint, params=params)
    activities_data = response.json()
    
    # Extract the activities
    activities = []
    for activity in activities_data['data']:
        if any(park['parkCode'] == params['parkCode'] for park in activity['parks']):
            activities.append(activity['name'])
    
    # Remove duplicates
    unique_activities = list(set(activities))
    
    return unique_activities